# memstack-agent-ui 前后端集成

本 notebook 演示如何将前端框架与后端 Agent 集成。

## 概述

集成要点：
- WebSocket 连接管理
- 事件流处理
- 状态同步
- HITL 交互

## 1. 项目结构

In [ ]:
# 推荐的项目结构

project_structure = '''
my-agent-app/
├── backend/
│   ├── main.py              # FastAPI 入口
│   ├── agent/
│   │   ├── __init__.py
│   │   ├── tools.py         # 使用 memstack-agent
│   │   └── processor.py
│   └── requirements.txt
│
├── frontend/
│   ├── src/
│   │   ├── App.tsx          # 使用 @memstack-agent-ui/react
│   │   ├── components/
│   │   │   ├── Chat.tsx
│   │   │   └── Timeline.tsx
│   │   └── main.tsx
│   ├── package.json
│   └── vite.config.ts
│
└── docker-compose.yml
'''

print(project_structure)

## 2. 后端 WebSocket 端点

In [ ]:
# FastAPI WebSocket 端点

backend_websocket = '''
# backend/main.py
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from fastapi.middleware.cors import CORSMiddleware
import json
import asyncio

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.websocket("/agent/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    
    # 获取查询参数
    session_id = websocket.query_params.get("session_id", "unknown")
    token = websocket.query_params.get("token")
    
    # TODO: 验证 token
    
    try:
        while True:
            # 接收消息
            data = await websocket.receive_text()
            message = json.loads(data)
            
            if message.get("type") == "ping":
                # 心跳响应
                await websocket.send_json({"type": "pong"})
            
            elif message.get("type") == "chat":
                # 处理聊天消息
                conversation_id = message.get("conversation_id")
                user_message = message.get("message")
                
                # 发送事件流
                async for event in process_agent_message(conversation_id, user_message):
                    await websocket.send_json(event)
    
    except WebSocketDisconnect:
        print(f"Client disconnected: {session_id}")


async def process_agent_message(conversation_id: str, message: str):
    """处理 Agent 消息并生成事件流。"""
    # 使用 memstack-agent 框架
    from memstack_agent import ThoughtEvent, ActEvent, ObserveEvent, TextDeltaEvent, CompleteEvent
    import time
    
    # 思考事件
    yield ThoughtEvent(
        conversation_id=conversation_id,
        content=f"Processing: {message}"
    ).to_dict()
    
    # 模拟工具调用
    yield ActEvent(
        conversation_id=conversation_id,
        tool_name="search",
        tool_input={"query": message}
    ).to_dict()
    
    await asyncio.sleep(0.5)  # 模拟处理
    
    yield ObserveEvent(
        conversation_id=conversation_id,
        tool_name="search",
        result="Found relevant information..."
    ).to_dict()
    
    # 流式文本响应
    response = "Based on my analysis, here is the answer..."
    for word in response.split():
        yield TextDeltaEvent(
            conversation_id=conversation_id,
            delta=word + " "
        ).to_dict()
        await asyncio.sleep(0.05)
    
    # 完成事件
    yield CompleteEvent(
        conversation_id=conversation_id,
        result=response,
        tokens={"total": 100},
        cost=0.001
    ).to_dict()
'''

print(backend_websocket)

## 3. 前端应用

In [ ]:
# 前端 App 组件

frontend_app = '''
// frontend/src/App.tsx
import { AgentProvider } from '@memstack-agent-ui/react';
import { ChatInterface } from './components/Chat';

function App() {
  // 从环境变量获取配置
  const wsUrl = import.meta.env.VITE_WS_URL || 'ws://localhost:8000/agent/ws';
  const token = localStorage.getItem('token') || undefined;
  
  // 从 URL 获取 conversation_id
  const urlParams = new URLSearchParams(window.location.search);
  const conversationId = urlParams.get('conversation') || 'default';
  
  return (
    <AgentProvider
      wsUrl={wsUrl}
      token={token}
      conversationId={conversationId}
    >n      <div className="app">
        <header>
          <h1>Agent Chat</h1>
        </header>
        <main>
          <ChatInterface conversationId={conversationId} />
        </main>
      </div>
    </AgentProvider>
  );
}

export default App;
'''

print(frontend_app)

## 4. 聊天组件

In [ ]:
# 聊天界面组件

chat_component = '''
// frontend/src/components/Chat.tsx
import { useState } from 'react';
import {
  useAgentChat,
  useConversation,
  useStreaming,
  ExecutionTimeline,
} from '@memstack-agent-ui/react';

interface ChatProps {
  conversationId: string;
}

export function ChatInterface({ conversationId }: ChatProps) {
  const [input, setInput] = useState('');

  // 聊天 hook
  const { submit, isRunning, status, error } = useAgentChat({
    wsUrl: import.meta.env.VITE_WS_URL,
    conversationId,
    onComplete: (data) => {
      console.log('Agent completed:', data);
    },
  });

  // 会话状态
  const { timeline, messages, agentState } = useConversation({
    conversationId,
  });

  // 流式状态
  const {
    isStreaming,
    streamingAssistantContent,
    streamingThought,
    activeToolCalls,
  } = useStreaming({ conversationId });

  const handleSubmit = () => {
    if (input.trim() && !isRunning) {
      submit(input.trim());
      setInput('');
    }
  };

  return (
    <div className="chat-interface">
      {/* 状态栏 */}
      <div className="status-bar">
        <span className={`status-dot ${status}`} />
        <span>{status}</span>
        {agentState !== 'idle' && <span> · {agentState}</span>}
      </div>

      {/* 主内容区 */}
      <div className="content">
        {/* 消息列表 */}
        <div className="messages">
          {messages.map((msg) => (
            <div key={msg.id} className={`message ${msg.role}`}>
              <div className="role">{msg.role}</div>
              <div className="content">{msg.content}</div>
            </div>
          ))}

          {/* 流式思考 */}
          {streamingThought && (
            <div className="message assistant thinking">
              <div className="role">thinking</div>
              <div className="content">{streamingThought}</div>
            </div>
          )}

          {/* 流式响应 */}
          {streamingAssistantContent && (
            <div className="message assistant streaming">
              <div className="role">assistant</div>
              <div className="content">{streamingAssistantContent}</div>
            </div>
          )}
        </div>

        {/* 执行时间线 */}
        <div className="timeline">
          <h3>Execution Timeline</h3>
          <ExecutionTimeline timeline={timeline} />
        </div>
      </div>

      {/* 输入区 */}
      <div className="input-area">
        <input
          type="text"
          value={input}
          onChange={(e) => setInput(e.target.value)}
          onKeyDown={(e) => e.key === 'Enter' && handleSubmit()}
          disabled={isRunning}
          placeholder="Type a message..."
        />
        <button onClick={handleSubmit} disabled={isRunning || !input.trim()}>
          {isStreaming ? '...' : 'Send'}
        </button>
        {error && <div className="error">{error.message}</div>}
      </div>
    </div>
  );
}
'''

print(chat_component)

## 5. Docker Compose 配置

In [ ]:
# Docker Compose 配置

docker_compose = '''
# docker-compose.yml
version: '3.8'

services:
  backend:
    build: ./backend
    ports:
      - "8000:8000"
    environment:
      - OPENAI_API_KEY=${OPENAI_API_KEY}
      - LOG_LEVEL=INFO
    volumes:
      - ./backend:/app

  frontend:
    build: ./frontend
    ports:
      - "3000:3000"
    environment:
      - VITE_WS_URL=ws://localhost:8000/agent/ws
    depends_on:
      - backend
    volumes:
      - ./frontend/src:/app/src
'''

print(docker_compose)

## 6. 事件流协议

In [ ]:
# 事件流协议定义

event_protocol = '''
// 客户端 -> 服务器消息

// 心跳
{ "type": "ping" }

// 聊天消息
{
  "type": "chat",
  "conversation_id": "conv-123",
  "message": "Hello, agent!",
  "file_ids": ["file-001"]  // 可选
}

// 订阅会话
{
  "type": "subscribe",
  "conversation_id": "conv-123"
}


// 服务器 -> 客户端事件

// 心跳响应
{ "type": "pong" }

// 思考事件
{
  "type": "thought",
  "conversation_id": "conv-123",
  "data": { "content": "I need to search for information..." },
  "timestamp": "2024-01-15T10:00:00Z"
}

// 工具调用事件
{
  "type": "act",
  "conversation_id": "conv-123",
  "data": {
    "tool_name": "search_web",
    "tool_input": { "query": "Python asyncio" }
  },
  "timestamp": "2024-01-15T10:00:01Z"
}

// 工具结果事件
{
  "type": "observe",
  "conversation_id": "conv-123",
  "data": {
    "tool_name": "search_web",
    "result": "Found 10 results...",
    "duration_ms": 350
  },
  "timestamp": "2024-01-15T10:00:02Z"
}

// 文本增量事件
{
  "type": "text_delta",
  "conversation_id": "conv-123",
  "data": { "delta": "Based on " },
  "timestamp": "2024-01-15T10:00:03Z"
}

// 完成事件
{
  "type": "complete",
  "conversation_id": "conv-123",
  "data": {
    "result": "Done",
    "tokens": { "total": 500 },
    "cost": 0.01
  },
  "timestamp": "2024-01-15T10:00:05Z"
}

// 错误事件
{
  "type": "error",
  "conversation_id": "conv-123",
  "data": {
    "message": "API rate limit exceeded",
    "code": "RATE_LIMIT"
  },
  "timestamp": "2024-01-15T10:00:05Z"
}
'''

print(event_protocol)

## 7. 错误处理

In [ ]:
# 完整的错误处理

error_handling = '''
import { useAgentChat } from '@memstack-agent-ui/react';

function ChatWithErrorHandling() {
  const [input, setInput] = useState('');
  const [errors, setErrors] = useState<string[]>([]);

  const { submit, isRunning, status, error } = useAgentChat({
    wsUrl: 'ws://localhost:8000/agent/ws',
    conversationId: 'conv-123',
    
    onError: (err) => {
      // WebSocket 连接错误
      setErrors((prev) => [...prev, `Connection error: ${err.message}`]);
    },
    
    onAgentError: (data) => {
      // Agent 执行错误
      const message = (data as any).message || 'Unknown error';
      setErrors((prev) => [...prev, `Agent error: ${message}`]);
    },
    
    onComplete: () => {
      // 成功完成时清除错误
      setErrors([]);
    },
  });

  const handleSubmit = async () => {
    if (!input.trim()) return;
    
    try {
      submit(input.trim());
      setInput('');
    } catch (err) {
      setErrors((prev) => [...prev, `Submit error: ${err}`]);
    }
  };

  return (
    <div>
      {/* 显示错误 */}
      {errors.length > 0 && (
        <div className="errors">
          {errors.map((err, i) => (
            <div key={i} className="error">
              {err}
              <button onClick={() => setErrors(errors.filter((_, j) => j !== i))}>
                ×
              </button>
            </div>
          ))}
        </div>
      )}

      {/* 连接状态 */}
      {status === 'error' && (
        <div className="connection-error">
          Connection failed. Trying to reconnect...
        </div>
      )}

      <input
        value={input}
        onChange={(e) => setInput(e.target.value)}
        disabled={status !== 'connected' || isRunning}
      />
      <button
        onClick={handleSubmit}
        disabled={status !== 'connected' || isRunning || !input.trim()}
      >
        Send
      </button>
    </div>
  );
}
'''

print(error_handling)

## 8. HITL 交互

In [ ]:
# Human-in-the-Loop 交互处理

hitl_handling = '''
import { useConversation } from '@memstack-agent-ui/react';

function HITLHandler({ conversationId }: { conversationId: string }) {
  const {
    pendingClarification,
    pendingDecision,
    pendingPermission,
    pendingHITLSummary,
  } = useConversation({ conversationId });

  // 处理澄清请求
  if (pendingClarification) {
    return (
      <ClarificationDialog
        request={pendingClarification}
        onAnswer={(answer) => {
          // 发送答案到后端
          submitHITLResponse({
            type: 'clarification_answered',
            request_id: pendingClarification.request_id,
            answer,
          });
        }}
      />
    );
  }

  // 处理决策请求
  if (pendingDecision) {
    return (
      <DecisionDialog
        request={pendingDecision}
        onDecision={(decision) => {
          submitHITLResponse({
            type: 'decision_answered',
            request_id: pendingDecision.request_id,
            decision,
          });
        }}
      />
    );
  }

  // 处理权限请求
  if (pendingPermission) {
    return (
      <PermissionDialog
        request={pendingPermission}
        onGrant={(granted) => {
          submitHITLResponse({
            type: 'permission_replied',
            request_id: pendingPermission.request_id,
            granted,
          });
        }}
      />
    );
  }

  return null;
}


// 权限对话框组件
function PermissionDialog({ request, onGrant }) {
  return (
    <div className="permission-dialog">
      <h3>Permission Required</h3>
      <p>The agent wants to use: <strong>{request.tool_name}</strong></p>
      <p>{request.patterns?.join(', ')}</p>
      <div className="actions">
        <button onClick={() => onGrant(true)}>Allow</button>
        <button onClick={() => onGrant(false)}>Deny</button>
      </div>
    </div>
  );
}
'''

print(hitl_handling)

## 总结

本 notebook 演示了前后端集成的完整流程：

1. **项目结构** - 推荐的目录组织
2. **后端 WebSocket** - FastAPI WebSocket 端点
3. **前端应用** - AgentProvider 包装
4. **聊天组件** - 使用 hooks 管理状态
5. **Docker Compose** - 开发环境配置
6. **事件协议** - 客户端-服务器消息格式
7. **错误处理** - 完整的错误处理模式
8. **HITL 交互** - Human-in-the-Loop 处理

### 快速启动

```bash
# 启动后端
cd backend && uvicorn main:app --reload

# 启动前端
cd frontend && pnpm dev

# 或使用 Docker Compose
docker-compose up
```